## Pysummury を試してみる。

In [ ]:
!pip install pysummarization
!pip install mecab-python3
!pip install unidic-lite
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import

In [ ]:
from pysummarization.nlpbase.auto_abstractor import AutoAbstractor
from pysummarization.tokenizabledoc.mecab_tokenizer import MeCabTokenizer
from pysummarization.abstractabledoc.top_n_rank_abstractor import TopNRankAbstractor

### MeCab
Mecabに大量の文章を読ませると分かち書きができないので、分割するように改変。当該部分だけ弄ろうとしたらなぜかできないので丸々書き換え。（誰か良い方法を教えてください）

In [ ]:
from pysummarization.tokenizable_doc import TokenizableDoc
import MeCab

class MeCabTokenizer(TokenizableDoc):
    '''
    Tokenize string.
    
    Japanese morphological analysis with MeCab.
    '''

    # Path ot mecab dictionary.
    # For instance, "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd".
    # If empty(""), this class will see default settings.
    __mecab_system_dic = ""

    def get_mecab_system_dic(self):
        ''' getter '''
        return self.__mecab_system_dic
    
    def set_mecab_system_dic(self, value):
        ''' setter '''
        self.__mecab_system_dic = value

    mecab_system_dic = property(get_mecab_system_dic, set_mecab_system_dic)

    __part_of_speech = []

    def get_part_of_speech(self):
        ''' getter '''
        return self.__part_of_speech
    
    def set_part_of_speech(self, value):
        ''' setter '''
        self.__part_of_speech = value

    part_of_speech = property(get_part_of_speech, set_part_of_speech)

    def tokenize(self, sentence_str):
        '''
        Tokenize str.
        
        Args:
            sentence_str:   tokenized string.
        
        Returns:
            [token, token, token, ...]
        '''
        if len(self.part_of_speech) == 0:
            mt = MeCab.Tagger(self.mecab_system_dic + " -Owakati")
            token_list = []
            chunks = sentence_str.split("\n")
            for p_text in chunks:
              wordlist = mt.parse(p_text)
              token_list += wordlist.rstrip(" \n").split(" ")
            return token_list
        else:
            tagger = MeCab.Tagger(self.mecab_system_dic + " -Ochasen")
            node = tagger.parseToNode(sentence_str)

            result_tuple_list = []
            token_list = []
            while node:
                feature_list = node.feature.split(",")
                if feature_list[0] != "BOS/EOS":
                    if feature_list[0] in self.part_of_speech:
                        token = feature_list[6]
                        token_list.append(token)
                node = node.next
            return token_list

### 要約のテスト

文章は
https://ja.wikipedia.org/wiki/%E6%98%A0%E7%94%BB
から一部を利用しました。

In [ ]:
def get_summary(document):
  auto_abstractor = AutoAbstractor()
  auto_abstractor.tokenizable_doc = MeCabTokenizer()
  auto_abstractor.delimiter_list = ["。", "\n"]
  abstractable_doc = TopNRankAbstractor()
  result_dict = auto_abstractor.summarize(document, abstractable_doc)
  return result_dict

result_dict = get_summary( """
19世紀後半に写真技術が発展すると、やがてそれを利用して動く写真の開発が始まり、1893年にトーマス・エジソンが1人でのぞき込んで楽しむキネトスコープを発明するなど、1890年代にはいくつかの映画の原型が考案されていた[5]。そうした中、1895年にフランスのリュミエール兄弟がスクリーンに動く写真を投影して公開した。これが現代にまでつながる映画の起源とされている[6]。
スクリーンに上映する映画は登場と同時に世界中で反響を呼び、開発の翌年には各国で上映されるようになった[5]。草創期の映画は単に事実を記録した映像に過ぎなかったが、それでも新奇さから各地の見世物で大当たりを取り、映画館が相次いで各地に設立された[7]。20世紀に入るとストーリーを持つ映画の制作が始まり、盛んに映画作品が作られるようになった[8]。
映画表現において大きな画期となったのは、1920年代の「トーキー」の登場、それに続いて行われたいわゆる「総天然色」映画の登場が数えられよう。これらはそれぞれ、それまでの映画の形式を最終的には駆逐するにいたった。例えば、今では「トーキー」以前の形式である「サイレント」が新たに発表されることはほぼない。また、今「モノクローム」で撮影された映画が発表されることは極めてまれである。
20世紀前半に行われたこれらの映画技術の進展とは異なり、20世紀後半の映画技術の発展は映画表現の多様性を増す方向に作用した。
戦後、普及した映画の撮影技法には、例えば「特殊撮影」「アニメーション」「コンピュータ・グラフィクス」が挙げられる。これらの新たな撮影技法は、それ以前の方法を駆逐することによって普及したのではなく、それが登場する以前の撮影技法と共存しつつ独自の分野を成す形でそれぞれの発展を遂げている。
1970年代からはVTRが普及したが、フィルムとビデオとの基本的な表示方式の違いから映画は35mmフィルムによる撮影が一般的であった。21世紀に入った頃から商業作品もデジタルビデオカメラで撮影され、フィルムを使わずコンピュータ上で編集される例が増加している。詳しくはデジタルシネマを参照。
1990年代以降はコンピューターを使って画像を生成したコンピューターグラフィックス通称CGが大々的に使われるようになる。""")
result_dict

{'summarize_result': ['19世紀後半に写真技術が発展すると、やがてそれを利用して動く写真の開発が始まり、1893年にトーマス・エジソンが1人でのぞき込んで楽しむキネトスコープを発明するなど、1890年代にはいくつかの映画の原型が考案されていた[5]。\n',
  'そうした中、1895年にフランスのリュミエール兄弟がスクリーンに動く写真を投影して公開した。\n',
  'これが現代にまでつながる映画の起源とされている[6]。\n',
  'スクリーンに上映する映画は登場と同時に世界中で反響を呼び、開発の翌年には各国で上映されるようになった[5]。\n',
  '草創期の映画は単に事実を記録した映像に過ぎなかったが、それでも新奇さから各地の見世物で大当たりを取り、映画館が相次いで各地に設立された[7]。\n',
  '20世紀に入るとストーリーを持つ映画の制作が始まり、盛んに映画作品が作られるようになった[8]。\n',
  '映画表現において大きな画期となったのは、1920年代の「トーキー」の登場、それに続いて行われたいわゆる「総天然色」映画の登場が数えられよう。\n',
  'これらはそれぞれ、それまでの映画の形式を最終的には駆逐するにいたった。\n',
  '20世紀前半に行われたこれらの映画技術の進展とは異なり、20世紀後半の映画技術の発展は映画表現の多様性を増す方向に作用した。\n',
  '1970年代からはVTRが普及したが、フィルムとビデオとの基本的な表示方式の違いから映画は35mmフィルムによる撮影が一般的であった。\n'],
 'scoring_data': [(0, 39.40909090909091),
  (1, 18.615384615384617),
  (2, 18.0),
  (3, 24.02857142857143),
  (4, 27.0),
  (5, 7.142857142857143),
  (6, 6.230769230769231),
  (7, 7.3478260869565215),
  (10, 8.047619047619047),
  (13, 6.25)]}

In [ ]:
result_dict["summarize_result"]

['19世紀後半に写真技術が発展すると、やがてそれを利用して動く写真の開発が始まり、1893年にトーマス・エジソンが1人でのぞき込んで楽しむキネトスコープを発明するなど、1890年代にはいくつかの映画の原型が考案されていた[5]。\n',
 'そうした中、1895年にフランスのリュミエール兄弟がスクリーンに動く写真を投影して公開した。\n',
 'これが現代にまでつながる映画の起源とされている[6]。\n',
 'スクリーンに上映する映画は登場と同時に世界中で反響を呼び、開発の翌年には各国で上映されるようになった[5]。\n',
 '草創期の映画は単に事実を記録した映像に過ぎなかったが、それでも新奇さから各地の見世物で大当たりを取り、映画館が相次いで各地に設立された[7]。\n',
 '20世紀に入るとストーリーを持つ映画の制作が始まり、盛んに映画作品が作られるようになった[8]。\n',
 '映画表現において大きな画期となったのは、1920年代の「トーキー」の登場、それに続いて行われたいわゆる「総天然色」映画の登場が数えられよう。\n',
 'これらはそれぞれ、それまでの映画の形式を最終的には駆逐するにいたった。\n',
 '20世紀前半に行われたこれらの映画技術の進展とは異なり、20世紀後半の映画技術の発展は映画表現の多様性を増す方向に作用した。\n',
 '1970年代からはVTRが普及したが、フィルムとビデオとの基本的な表示方式の違いから映画は35mmフィルムによる撮影が一般的であった。\n']

In [ ]:
result_dict["summarize_result"][0:5]

['19世紀後半に写真技術が発展すると、やがてそれを利用して動く写真の開発が始まり、1893年にトーマス・エジソンが1人でのぞき込んで楽しむキネトスコープを発明するなど、1890年代にはいくつかの映画の原型が考案されていた[5]。\n',
 'そうした中、1895年にフランスのリュミエール兄弟がスクリーンに動く写真を投影して公開した。\n',
 'これが現代にまでつながる映画の起源とされている[6]。\n',
 'スクリーンに上映する映画は登場と同時に世界中で反響を呼び、開発の翌年には各国で上映されるようになった[5]。\n',
 '草創期の映画は単に事実を記録した映像に過ぎなかったが、それでも新奇さから各地の見世物で大当たりを取り、映画館が相次いで各地に設立された[7]。\n']

### 大量のテキストで実験
むちゃを承知で、三国志の全テキストを投入。

In [ ]:
txt_path = '/content/train.txt'

with open(txt_path, mode='r', encoding='UTF-8') as f:
  document = f.read()

print(type(document))

<class 'str'>


In [ ]:
result_dict = get_summary(document)
result_dict

{'summarize_result': ['今から約千七百八十年ほど前のことである。\n',
  '腰に、一剣を佩いているほか、身なりはいたって見すぼらしいが、眉は秀で、唇は紅く、とりわけ聡明そうな眸や、豊かな頬をしていて、つねにどこかに微笑をふくみ、総じて賤しげな容子がなかった。\n',
  '草むらの中に、ぽつねんと坐って、膝をかかえこんでいた。\n',
  'そしてそこは、黄河の畔の黄土層の低い断り岸であった。\n',
  'あなたが佩いている剣は誰にもらったのか」\n',
  '猟師にしては、どこか骨柄の秀でたところが見えたからである。\n',
  '予期していたことではあるが、そう聞くと、みな愕然とした色を顔にたたえた。\n',
  'だが、誰あって、進んで彼の首に剣を加えようとする者はなかった。\n',
  'すると一河の水に、頑丈な木橋が架かっていた。\n',
  'むしろ幕下の大将がかくまで旺盛な戦意を抱いていることは彼としてよろこばしいほどであったから、\n'],
 'scoring_data': [(2, 14.0),
  (4, 28.491525423728813),
  (6, 15.058823529411764),
  (10, 14.222222222222221),
  (431, 11.076923076923077),
  (17003, 11.842105263157896),
  (20719, 11.076923076923077),
  (24398, 11.130434782608695),
  (30478, 10.5625),
  (40684, 10.24)]}

In [ ]:
result_dict["summarize_result"]

['今から約千七百八十年ほど前のことである。\n',
 '腰に、一剣を佩いているほか、身なりはいたって見すぼらしいが、眉は秀で、唇は紅く、とりわけ聡明そうな眸や、豊かな頬をしていて、つねにどこかに微笑をふくみ、総じて賤しげな容子がなかった。\n',
 '草むらの中に、ぽつねんと坐って、膝をかかえこんでいた。\n',
 'そしてそこは、黄河の畔の黄土層の低い断り岸であった。\n',
 'あなたが佩いている剣は誰にもらったのか」\n',
 '猟師にしては、どこか骨柄の秀でたところが見えたからである。\n',
 '予期していたことではあるが、そう聞くと、みな愕然とした色を顔にたたえた。\n',
 'だが、誰あって、進んで彼の首に剣を加えようとする者はなかった。\n',
 'すると一河の水に、頑丈な木橋が架かっていた。\n',
 'むしろ幕下の大将がかくまで旺盛な戦意を抱いていることは彼としてよろこばしいほどであったから、\n']

### Similarity Filter


In [ ]:
from pysummarization.nlp_base import NlpBase
from pysummarization.similarityfilter.tfidf_cosine import TfIdfCosine

def get_summary_2(document) -> dict:
  similarity_limit = 0.05
  nlp_base = NlpBase()
  nlp_base.tokenizable_doc = MeCabTokenizer()
  similarity_filter = TfIdfCosine()
  similarity_filter.nlp_base = nlp_base
  similarity_filter.similarity_limit = similarity_limit
  auto_abstractor = AutoAbstractor()
  auto_abstractor.tokenizable_doc = MeCabTokenizer()
  auto_abstractor.delimiter_list = ["。", "\n"]
  abstractable_doc = TopNRankAbstractor()
  result_dict2 = auto_abstractor.summarize(document, abstractable_doc, similarity_filter)
  return result_dict2


In [ ]:
result_dict2 = get_summary_2(document)

/usr/local/lib/python3.7/dist-packages/pysummarization/similarityfilter/tfidf_cosine.py:46: RuntimeWarning: invalid value encountered in double_scalars
  result = dot_prod / (norm_x * norm_y)


In [ ]:
# 出力
result_dict2["summarize_result"]

['黄巾賊\n', '今から約千七百八十年ほど前のことである。\n', '独り強くそう思う。\n', 'ただ追い詰め追い詰め討ちくずせ」\n']

## 参考

公式
- https://code.accel-brain.com/Automatic-Summarization/

解説
- https://kiyosucyberclub.web.fc2.com/DeZero_NLP/DeZero_VC05-13D.html
- https://resanaplaza.com/2022/05/19/%E3%80%90%E5%AE%9F%E8%B7%B5%E3%80%91python%EF%BC%8Bpysummarization%E3%81%A7%E6%96%87%E6%9B%B8%E8%A6%81%E7%B4%84%EF%BC%88%E3%83%86%E3%82%AD%E3%82%B9%E3%83%88%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3/
- https://book.st-hakky.com/docs/text-summary-library-pysummarization
